In [1]:
import cvxpy as cp
import numpy as np
np.random.seed(0x364A_23F2)
N = 500
n = 10
X = np.random.uniform(low=-1, high=1, size=(N, n))
y = np.zeros(N)

_c = np.random.randn(n)
_d = np.random.randn(n)

#We generate data as gaussians with mean and variance dependent on the feature x.
for i in range(N):
    y[i] = np.random.normal(loc=_c.T @ X[i], scale=abs(_d.T @ X[i]))

In [2]:
def phi(j,u):
  return (1/2) * cp.abs(u) + (1/2 - 0.1*(j+1))*u
def difference(v,teta,x,y):
  return v+ teta*x-y

In [3]:
v = cp.Variable((9,1))
teta = cp.Variable((9,n))
objectivbe = cp.Minimize(cp.sum(cp.vstack([cp.sum(cp.vstack([phi(i,difference(v[i],teta[i,:], X[t,:], y[t])) for i in range(9)]))for t in range(0,N)])))

Streaming output truncated to the last 5000 lines.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 3945 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 3946 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and

In [4]:
objectivbe

Minimize(Expression(CONVEX, UNKNOWN, ()))

In [5]:
constraints = []
for i in range(8):
  constraints += [cp.norm((teta[i+1,:]-teta[i,:]),1)<= v[i+1]-v[i]]



In [6]:
constraints

[Inequality(Expression(CONVEX, NONNEGATIVE, ())),
 Inequality(Expression(CONVEX, NONNEGATIVE, ())),
 Inequality(Expression(CONVEX, NONNEGATIVE, ())),
 Inequality(Expression(CONVEX, NONNEGATIVE, ())),
 Inequality(Expression(CONVEX, NONNEGATIVE, ())),
 Inequality(Expression(CONVEX, NONNEGATIVE, ())),
 Inequality(Expression(CONVEX, NONNEGATIVE, ())),
 Inequality(Expression(CONVEX, NONNEGATIVE, ()))]

In [7]:
problem = cp.Problem(objectivbe,constraints)
result = problem.solve()

/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:155: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


In [12]:
print(result)
print(v.value)
print(teta.value[0])

3129.3211024692837
[[-2.54413669]
 [-1.34728588]
 [-0.74604442]
 [-0.27686327]
 [-0.03760847]
 [ 0.25808255]
 [ 0.88644751]
 [ 1.48845713]
 [ 2.71852354]]
[ 0.53330889 -1.77468807 -1.31942949  0.1528987   1.44823285 -0.13765922
 -0.17844532  1.54278969  0.09559465  1.06927663]


In [14]:
q = v.value + (teta.value)@(X.T)
print('\n'.join([f"Fraction y <= q_{j+1}: {(np.sum(q[j] >= y)) / N}"for j in range(9)]))


Fraction y <= q_1: 0.1
Fraction y <= q_2: 0.2
Fraction y <= q_3: 0.298
Fraction y <= q_4: 0.4
Fraction y <= q_5: 0.496
Fraction y <= q_6: 0.6
Fraction y <= q_7: 0.696
Fraction y <= q_8: 0.798
Fraction y <= q_9: 0.902


Part C

In [9]:
problem_woc = cp.Problem(objectivbe)
result_woc = problem_woc.solve()
v_tilde = v.value
teta_tilde = teta.value
teta_tilde_1 = teta.value[0]
print(result_woc)
print(v_tilde)
print(teta_tilde_1)

/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:155: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


3122.4294751964862
[[-2.54413669]
 [-1.34728588]
 [-0.74604442]
 [-0.27686327]
 [-0.03760847]
 [ 0.25808255]
 [ 0.88644751]
 [ 1.48845713]
 [ 2.71852354]]
[ 0.53330889 -1.77468807 -1.31942949  0.1528987   1.44823285 -0.13765922
 -0.17844532  1.54278969  0.09559465  1.06927663]


In [18]:
q_tilde = v_tilde + teta_tilde @ X.T
print(q_tilde.shape)
print(q_tilde[:,423])
print(X[423,:])
print('this is the x_viol that we wanted to find and corresponding q')

(9, 500)
[-2.3501676  -2.19644457 -1.63645953 -0.91770962 -0.96613347 -0.73859306
 -0.07749717  0.47638803  2.25006082]
[ 0.50662982  0.72030365 -0.31042263 -0.80958473  0.55994599  0.84563168
  0.72929439  0.32341836  0.34966115 -0.16878643]
this is the x_viol that we wanted to find and corresponding q


In [11]:
print(np.ediff1d(q_tilde[:,423]))

[ 0.15372302  0.55998504  0.71874991 -0.04842385  0.22754041  0.6610959
  0.5538852   1.77367279]


In [16]:
print(np.ediff1d(q_tilde[:,423])[3])

-0.048423845792634146


In [17]:
print('the difference between q4 and q5 is positive')

the difference between q4 and q5 is positive
